In [1]:
import numpy as np
import pandas as pd
from ta import*
import matplotlib.pyplot as plt
#import seaborn 
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import math
#import seaborn as sns


In [2]:
data = pd.read_csv('SPEND_NEW.csv')
data = data.dropna()
data = data.drop(['PER','DATE','TICKER','TIME'],axis=1)
data.head()

,OPEN,HIGH,LOW,CLOSE,VOL
0,2105.23,2109.90,2104.50,2109.63,1349
1,2109.68,2113.28,2109.68,2113.21,1094
2,2113.45,2113.97,2111.83,2112.88,1100
3,2112.89,2113.21,2111.32,2112.36,1060
4,2112.35,2114.38,2112.11,2112.28,944


In [3]:
data.count()

OPEN     13853
HIGH     13853
LOW      13853
CLOSE    13853
VOL      13853
dtype: int64

In [4]:
data.describe()

,OPEN,HIGH,LOW,CLOSE,VOL
count,13853.000000,13853.000000,13853.000000,13853.000000,13853.000000
mean,2355.344382,2357.778322,2352.816145,2355.351823,1206.078250
std,296.480873,296.641664,296.241452,296.465423,397.099395
min,1813.050000,1820.280000,1810.100000,1813.080000,1.000000
25%,2093.090000,2095.150000,2091.140000,2093.110000,1104.000000
50%,2334.470000,2336.410000,2333.470000,2334.880000,1294.000000
75%,2644.360000,2649.060000,2639.720000,2643.850000,1459.000000
max,2937.570000,2940.910000,2936.570000,2937.460000,1756.000000


In [5]:
data.corr()

,OPEN,HIGH,LOW,CLOSE,VOL
OPEN,1.000000,0.999939,0.999925,0.999889,0.250156
HIGH,0.999939,1.000000,0.999874,0.999936,0.253978
LOW,0.999925,0.999874,1.000000,0.999940,0.245938
CLOSE,0.999889,0.999936,0.999940,1.000000,0.249946
VOL,0.250156,0.253978,0.245938,0.249946,1.000000


# working with Technical indicators for underlying the the trend of stock market data

In [6]:
#finding the moving average for every 40 minutes 
def moving_average(data,minutes):
    MA = pd.Series(data['CLOSE'].rolling(minutes,min_periods=minutes).mean(),name= 'MA')
    MA.dropna(axis=0,inplace=True)
    MA = MA.reset_index(drop=True)
    df = pd.DataFrame({'MA':MA})
    return df

In [7]:
#CALCULATIG A MOVING AVERAGE FOR  40MINUTES.
minutes = 40
ma = moving_average(data,40)
ma.head()

,MA
0,2105.24475
1,2104.97075
2,2104.60375
3,2104.34000
4,2104.12225


In [8]:
# This moving avrage is also use to analyze the trend movement of stock data.

def EMA(data,minutes):
    ema = pd.Series(data['CLOSE'].ewm(span= minutes,min_periods= minutes).mean(),name ='ema')
    ema.dropna(axis=0,inplace=True)
    ema =ema.reset_index(drop= True)
    df = pd.DataFrame({'ema':ema})
    return df

In [9]:
minutes  = 40
ema = EMA(data, 40)
ema.head()

,ema
0,2102.085507
1,2101.894292
2,2101.707290
3,2101.741669
4,2101.846352


In [10]:
def CCI(data, minutes,constant=0.015):
    
    TP =  (data['HIGH']+data['LOW']+data['CLOSE'])/3
    
    Sma = (TP)/minutes
    
    D = TP- Sma
    
    #for i in sum(D):
        
    sma_1 =sum (D)/minutes
        
    M = sma_1*0.015
        
    CCI = M/D
        
    CCI.dropna(axis =0, inplace =True)
           
    CCI = CCI.reset_index(drop=True)
                
    df = pd.DataFrame({"CCI":CCI})
    
            
    return df

In [11]:
cci= CCI(data,40)

cci.head()

,CCI
0,5.804322
1,5.793201
2,5.790907
3,5.792543
4,5.790825


In [12]:
# the aveerage true range which is also another trend analyze on difference and absolute values of high low and close prices of stock market data.
def ATR(data,minutes):
    
    ATR1 = abs (data['HIGH'] - data['LOW']).shift(minutes)
    ATR2 = abs (data['HIGH'] - data['CLOSE']).shift(minutes)
    ATR3 = abs (data['LOW'] - data['CLOSE']).shift(minutes)

    ATR = pd.DataFrame({'ATR1':ATR1, 'ATR2':ATR2, 'ATR3':ATR3})
    ATR = ATR.max(axis=1)
    ATR.dropna(axis=0,inplace=True)
    ATR = ATR.reset_index(drop =True)
    df = pd.DataFrame({'ATR':ATR})
    
    return df
    
    

In [13]:
minutes = 40

atr = ATR(data, 40)

atr.head()


,ATR
0,5.40
1,3.60
2,2.14
3,1.89
4,2.27


In [14]:
# Algorithm:

# momentum = (todays close price)- (close price ndays ago)

# WE NEED TO COMPUTE THE MOMENTUMM AND APPEND IT TO THE DATASET .

def Momentum(data, minutes):
    
    momentum = data['CLOSE'] - data['CLOSE'].shift(minutes)
    
    momentum.dropna(axis=0, inplace=True)
    
    momentum = momentum.reset_index(drop=True)
    
    df = pd.DataFrame({"momentum":momentum})
    
    print(df.head())
    
    return df
    

In [15]:
momentum = Momentum(data, 40)
momentum.head()

   momentum
0    -10.96
1    -14.68
2    -10.55
3     -8.71
4     -9.02


,momentum
0,-10.96
1,-14.68
2,-10.55
3,-8.71
4,-9.02


In [16]:
#the rate of change on the current and previous closing prices of stocks.

def ROC(data,minutes):
    
    M  = data['CLOSE'].diff(minutes-1)
    
    N = data['CLOSE'].shift(minutes-1)
    
    roc = pd.Series(((M/N)*100),name='roc'+str(minutes))
    
    roc.dropna(axis =0,inplace =True)
    
    roc = roc.reset_index(drop=True)
    
    df = pd.DataFrame({"roc":roc})
    
    print(df.head())
    
    return df

In [17]:
roc = ROC(data,40)
roc.head()

        roc
0 -0.534217
1 -0.688053
2 -0.679168
3 -0.474824
4 -0.408563


,roc
0,-0.534217
1,-0.688053
2,-0.679168
3,-0.474824
4,-0.408563


In [18]:
#Detrended Price Oscillator (DPO)
#Is an indicator designed to remove trend from price and make it easier to
    #identify cycles
    
def detrended_price(data, minutes):
    
    dpo =pd.Series( data['CLOSE'].shift(int((0.5 * minutes) + 1)) - data['CLOSE'].rolling(minutes).mean())
    
    dpo = dpo.reset_index(drop=True)
    
    dpo.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"dpo":dpo})
      
    return df

In [19]:
minutes = 40
dpo = detrended_price(data,40)
dpo.head()


,dpo
39,-3.37475
40,-5.34075
41,-0.29375
42,0.60000
43,0.26775


In [20]:
# the Force index(1) = {Close (current period) - Close (prior period)} x Current period Volume
# Force Index 
def ForceIndex(data, minutes):

    FI = pd.Series(data['CLOSE'].diff(minutes) * data['VOL'], name = 'ForceIndex')    
     
    FI = FI.reset_index(drop=True)
    
    FI.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"FI":FI})
      
    return df
    


In [21]:
minutes = 40

fi = ForceIndex(data,40)

fi.head()

,FI
40,-12680.72
41,-396.36
42,-10043.60
43,-9520.03
44,-10282.80


In [22]:
# A vortex indicator (VI) is an indicator composed of two lines - an uptrend line (VI+) and a downtrend line (VI-). ... A vortex indicator is used to spot trend reversals and confirm current 
# Vortex indicator negative
def vortex_indicator_neg(data,minutes):
    
    tr = data['HIGH'].combine(data['CLOSE'].shift(1), max) - data['LOW'].combine(data['CLOSE'].shift(1), min)
    trn = tr.rolling(minutes).sum()

    vmp = np.abs(data['HIGH']- data['LOW'].shift(1)) 
    
    vmm = np.abs(data['LOW']- data['HIGH'].shift(1))
    vin = pd.Series(vmm.rolling(minutes).sum() / tr, name='vortex_indicator_neg')
    vin = vin.reset_index(drop =True)
    vin.dropna(axis =0,inplace =True)
    df = pd.DataFrame({'vin':vin})
    return df

    

In [23]:
minutes = 40
vin = vortex_indicator_neg(data,40)
vin.head()

,vin
40,74.444444
41,1060.714286
42,31.741935
43,54.906367
44,110.871212


In [24]:
# vortex indicator positive
def vortex_indicator_pos(data,minutes):
    tr = data['HIGH'].combine(data['CLOSE'].shift(1), max) - data['LOW'].combine(data['CLOSE'].shift(1), min)   
    trn = tr.rolling(minutes).sum()
    vmp = np.abs(data['HIGH'] - data['LOW'].shift(1))
    vmm = np.abs(data['LOW']- data['HIGH'].shift(1))
    vip = pd.Series(vmp.rolling(minutes).sum() / trn, name='vortex_indicator_pos')
    vip = vip.reset_index(drop =True)
    vip.dropna(axis =0,inplace =True)
    df = pd.DataFrame({'vip':vip})
    return df

    
    
    


In [25]:
minutes = 40 
vip = vortex_indicator_pos(data,40)
vip.head()

,vip
40,0.957240
41,0.923105
42,0.905899
43,0.931692
44,0.938687


In [26]:
# standard deviation of the closing prices.
def standard_deviation(data, minutes):
    
    deviat = pd.Series(data['CLOSE'].rolling(minutes, min_periods=minutes).std(), name='STD_')
    
    
    deviat = deviat.reset_index(drop=True)
    
    deviat.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"deviat":deviat})
    
    return df

In [27]:
minutes = 40
deviat = standard_deviation(data,40)
deviat.head()

,deviat
39,7.229926
40,7.267059
41,7.210758
42,7.092244
43,6.972393


In [28]:
def mass_index(data, minutes,n_minutes):
    
    Range = data['HIGH'] - data['LOW']
    Ema = Range.ewm(minutes,min_periods=minutes).mean()
    Ema_ema = Ema.ewm(minutes, min_periods=minutes).mean()
    Mass = Ema / Ema_ema
    MassI = pd.Series(Mass.rolling(n_minutes).sum(), name='Mass Index')
    MassI  = MassI.reset_index(drop=True)
    
    MassI.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"MassI":MassI})

    return df

In [29]:
minutes = 30
n_minutes = 60
massi = mass_index(data,30,40)
massi.head()

,MassI
97,42.859735
98,42.950430
99,42.992770
100,43.046040
101,43.063957


In [30]:
# under normall circumstances this indicator works within  a span of 3days and min_periods of 10days 
# it applies exponential weight averages on accumulation distribution indicator and its differences.

def chaikin_oscillator(data, minutes,n_minutes ):
    
    
    ad = (2 * data['CLOSE'] - data['HIGH'] - data['LOW']) / (data['HIGH'] - data['LOW']) * data['VOL']
    
    Chaikin = pd.Series(ad.ewm(minutes, min_periods=minutes).mean() - ad.ewm(n_minutes, min_periods=n_minutes).mean(), name='Chaikin')
    
    Chaikin = Chaikin.reset_index(drop=True)
    
    Chaikin.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"Chaikin":Chaikin})
    
    return df


In [31]:
minutes = 30
n_minutes= 40
chai = chaikin_oscillator(data,30,40)
chai.head()


,Chaikin
39,-2.161177
40,-3.644025
41,-3.968413
42,0.064934
43,4.949201


In [32]:
#KST Oscillator  r is the rate period and m  is the smoothed period usual use in trend analysis.
def KST_oscillator(data, r1, r2, r3, r4, m_1, m_2, m_3, m_4):  
    M = data['CLOSE'].diff(r1 - 1)  
    N = data['CLOSE'].shift(r1 - 1)  
    ROC1 = M / N  
    M = data['CLOSE'].diff(r2 - 1)  
    N = data['CLOSE'].shift(r2 - 1)  
    ROC2 = M / N  
    M = data['CLOSE'].diff(r3 - 1)  
    N = data['CLOSE'].shift(r3 - 1)  
    ROC3 = M / N  
    M = data['CLOSE'].diff(r4 - 1)  
    N = data['CLOSE'].shift(r4 - 1)  
    ROC4 = M / N  
    KST = pd.Series(pd.Series(ROC1).rolling(m_1).sum() + pd.Series(ROC2).rolling(m_2).sum() * 2 + pd.Series(ROC3).rolling(m_3).sum() * 3 + pd.Series(ROC4).rolling(m_4).sum() * 4, name = 'KST_'  )
    KST  = KST.reset_index(drop=True)
    
    
    KST.dropna(axis=0,inplace=True) 
    
    df = pd.DataFrame({"KST":KST})
    
    return df

In [33]:
# the rate changings in every 40 and 50 minutes.
r1 =15
r2=20
r3=25
r4=30
m_1=40
m_2=40
m_3=40
m_4=50
KST = KST_oscillator(data,40,45,50,60,40,40,40,60)
KST.head()


,KST
118,-8.873236
119,-8.865349
120,-8.838247
121,-8.802236
122,-8.747278


In [34]:
# AROON UP AND AROON DOWN 
# AROON UP =UP-({N-DAY OF THE N-DAY PERIO OF HIGH}*N)*100
def aroon_up(data,minutes):
    aroon_up = data['CLOSE'].rolling(minutes).apply(lambda x:float(np.argmax(x)+1)/minutes*100)
    new_aroon = pd.Series(aroon_up)
    new_aroon  = new_aroon.reset_index(drop=True)
    new_aroon.dropna(axis=0,inplace=True) 
    df = pd.DataFrame({"new_aroon":new_aroon})
    return df

    

In [35]:
minutes=60
new_aroon = aroon_up(data,60)
new_aroon.head()

c:\users\1\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


,new_aroon
59,23.333333
60,21.666667
61,20.000000
62,18.333333
63,16.666667


In [36]:
#AROON DOW = DOWN -({N-DAY OF THE N-DAY PERIO OF LOW}*N)*100)

def aroon_down(data,minutes):
    aroon_down = data['CLOSE'].rolling(minutes).apply(lambda x:float(np.argmax(x)+1)/minutes*100)
    aroon_down = pd.Series(aroon_down)
    aroon_down = aroon_down.reset_index(drop=True)
    aroon_down.dropna(axis=0,inplace=True) 
    df = pd.DataFrame({"aroon_down":aroon_down})
    return df

    

In [37]:
minutes = 40
aroon_down = aroon_down(data,40)
aroon_down.head()

c:\users\1\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


,aroon_down
39,35.0
40,32.5
41,30.0
42,27.5
43,25.0


In [38]:
#  The TRIX Shows the percent rate of change of a triple exponentially smoothed moving
    #average.
    
def TRIX(data, minutes): 
    
    Ema = data['CLOSE'].ewm(span=minutes, min_periods=minutes).mean()
    Ema_2= Ema.ewm(span=minutes, min_periods=minutes).mean()
    Ema_3 = Ema_2.ewm(span=minutes, min_periods=minutes).mean()
    ROC = (Ema_3 - Ema_3.shift(1)) / Ema_3.shift(1)
    trix = pd.Series(ROC, name='trix_' )  
    trix = trix.reset_index(drop=True)
    trix.dropna(axis=0,inplace=True)  
    df = pd.DataFrame({"trix":trix})
    
    return df


In [39]:
minutes = 40 
trix = TRIX(data,40)
trix.head()

,trix
118,-0.000265
119,-0.000261
120,-0.000256
121,-0.000251
122,-0.000246


In [40]:
# It relates price and volume in the stock market. OBV is based on a
    #cumulative total volume.
def on_balance_volume(data, minutes):
 
    i = 0
 
    OBV = [0]
    # I have a dataframe with open high low close volume of stock prices and wanted to calaculate obv of parameter 10 and 20
    for i in range(len(data.index) - 1):
 
        if data.iloc[i + 1]['CLOSE'] - data.iloc[i]['CLOSE'] > 0:
            OBV.append(data.iloc[i]['VOL'])
        elif data.iloc[i + 1]['CLOSE'] - data.iloc[i]['CLOSE'] < 0:
            OBV.append(-data.iloc[i + 1]['VOL'])
        else:
            OBV.append(0)
 
    OBV = pd.Series(OBV)
 
    # obv is calculated till here but not passed to the rolling function
    OBV_ma = pd.Series(OBV.rolling(minutes, min_periods=minutes).mean(), name='OBV_')
    
    OBV_ma = OBV_ma.reset_index(drop=True)
    
    OBV_ma.dropna(axis=0,inplace=True)
    
    df = pd.DataFrame({"OBV":OBV})
    
    print(df.head())
    
    return df
    
 
    #data = data.join(OBV_ma)
 
    #return data
 
# obv.csv has the inputs
#aapl = pd.read_csv("obv.csv")
#n = 60
#print(on_balance_volume(data, n))

In [41]:
minutes = 40
OBV = on_balance_volume(data,40)

OBV.head()

      OBV
0     0.0
1  1349.0
2 -1100.0
3 -1060.0
4  -944.0


,OBV
0,0.0
1,1349.0
2,-1100.0
3,-1060.0
4,-944.0


In [42]:
#TRUE STRENGTH INDEX, s is the sensitive in time of how the market will look like and we choose 30 minutes and s= 40
def true_strength_index(data, minutes, s):
    
    M = pd.Series(data['CLOSE'].diff(1))
    aM = abs(M)
    EMA1 = pd.Series(M.ewm(span=minutes, min_periods=minutes).mean())
    aEMA1 = pd.Series(aM.ewm(span=minutes, min_periods=minutes).mean())
    EMA2 = pd.Series(EMA1.ewm(span=s, min_periods=s).mean())
    aEMA2 = pd.Series(aEMA1.ewm(span=s, min_periods=s).mean())
    TSI = pd.Series(EMA2 / aEMA2, name='TSI_' )
    TSI = TSI.reset_index(drop=True)
    TSI.dropna(axis=0,inplace=True)
    df = pd.DataFrame({"TSI":TSI})
    return df

In [43]:
minutes = 30
s = 40
Tsi = true_strength_index(data,30,40)
Tsi.head()

,TSI
69,-0.324603
70,-0.326546
71,-0.328850
72,-0.326261
73,-0.320728


In [44]:
# AVERAGE DIRECTIONAL INDEX.for trend analyze 
def ADX(data, minutes, n_ADX):
    
    i = 0
    UpI = []
    DoI = []
    while i + 1 <= data.index[-1]:
        UpMove = data.loc[i + 1, 'HIGH'] - data.loc[i, 'HIGH']
        DoMove = data.loc[i, 'LOW'] - data.loc[i + 1, 'LOW']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    i = 0
    TR_l = [0]
    while i < data.index[-1]:
        TR = max(data.loc[i + 1, 'HIGH'], data.loc[i, 'CLOSE']) - min(data.loc[i + 1, 'LOW'], data.loc[i, 'CLOSE'])
        TR_l.append(TR)
        i = i + 1
    TR_s = pd.Series(TR_l)
    ATR = pd.Series(TR_s.ewm(span=minutes, min_periods=minutes).mean())
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=minutes, min_periods=minutes).mean() / ATR)
    NegDI = pd.Series(DoI.ewm(span=minutes, min_periods=minutes).mean() / ATR)
    ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, min_periods=n_ADX).mean())
    ADX = ADX.reset_index(drop=True)
    ADX.dropna(axis=0,inplace=True)
    df = pd.DataFrame({'ADX':ADX})
    return df

In [45]:
minutes = 40
n_adx = 50

adx = ADX(data,40, 50 )
adx.head()

,ADX
88,0.343617
89,0.350591
90,0.354917
91,0.359026
92,0.364562


In [46]:
# we CONCANTENATE EMA , MOMNETUM,  ROC AND MACD
data.dropna(axis=0, inplace=True)

new_data = data.reset_index(drop=True)

df = pd.concat([new_data,ma,ema, momentum, roc,cci,atr,fi,deviat,massi,chai,adx,trix,vin,vip,Tsi,aroon_down,new_aroon,OBV], axis=1)

#df = df.loc[:,~df.columns.duplicated()]

#df.dropna(axis =0,inplace =True)

df = df.astype(float) 
#df = df.fillna(df.median(axis=0)
df = df.fillna(0)
df.head()


,OPEN,HIGH,LOW,CLOSE,VOL,MA,ema,momentum,roc,CCI,...,MassI,Chaikin,ADX,trix,vin,vip,TSI,aroon_down,new_aroon,OBV
0,2105.23,2109.90,2104.50,2109.63,1349.0,2105.24475,2102.085507,-10.96,-0.534217,5.804322,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2109.68,2113.28,2109.68,2113.21,1094.0,2104.97075,2101.894292,-14.68,-0.688053,5.793201,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1349.0
2,2113.45,2113.97,2111.83,2112.88,1100.0,2104.60375,2101.707290,-10.55,-0.679168,5.790907,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1100.0
3,2112.89,2113.21,2111.32,2112.36,1060.0,2104.34000,2101.741669,-8.71,-0.474824,5.792543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1060.0
4,2112.35,2114.38,2112.11,2112.28,944.0,2104.12225,2101.846352,-9.02,-0.408563,5.790825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-944.0


In [53]:
# scale the datapoint. using the z-score normalization fucnction

def normalized(df):
    normal_points = (df-df.min())/(df.max()-df.min())
    return normal_points

In [54]:
new_df = normalized(df)
new_df = new_df.fillna(0)
new_df.head()

,OPEN,HIGH,LOW,CLOSE,VOL,MA,ema,momentum,roc,CCI,...,MassI,Chaikin,ADX,trix,vin,vip,TSI,aroon_down,new_aroon,OBV
0,0.259826,0.258444,0.261347,0.263745,0.768091,0.718585,0.717667,0.523820,0.528978,0.636702,...,0.0,0.509085,0.0,0.690925,0.0,0.0,0.480805,0.0,0.0,0.500000
1,0.263784,0.261460,0.265946,0.266929,0.622792,0.718491,0.717602,0.513802,0.519473,0.632385,...,0.0,0.509085,0.0,0.690925,0.0,0.0,0.480805,0.0,0.0,0.884112
2,0.267136,0.262076,0.267854,0.266636,0.626211,0.718366,0.717538,0.524924,0.520022,0.631494,...,0.0,0.509085,0.0,0.690925,0.0,0.0,0.480805,0.0,0.0,0.186788
3,0.266638,0.261398,0.267402,0.266173,0.603419,0.718276,0.717550,0.529879,0.532648,0.632129,...,0.0,0.509085,0.0,0.690925,0.0,0.0,0.480805,0.0,0.0,0.198178
4,0.266158,0.262442,0.268103,0.266102,0.537322,0.718202,0.717586,0.529044,0.536743,0.631462,...,0.0,0.509085,0.0,0.690925,0.0,0.0,0.480805,0.0,0.0,0.231207


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13853 entries, 0 to 13852
Data columns (total 23 columns):
OPEN          13853 non-null float64
HIGH          13853 non-null float64
LOW           13853 non-null float64
CLOSE         13853 non-null float64
VOL           13853 non-null float64
MA            13853 non-null float64
ema           13853 non-null float64
momentum      13853 non-null float64
roc           13853 non-null float64
CCI           13853 non-null float64
ATR           13853 non-null float64
FI            13853 non-null float64
deviat        13853 non-null float64
MassI         13853 non-null float64
Chaikin       13853 non-null float64
ADX           13853 non-null float64
trix          13853 non-null float64
vin           13853 non-null float64
vip           13853 non-null float64
TSI           13853 non-null float64
aroon_down    13853 non-null float64
new_aroon     13853 non-null float64
OBV           13853 non-null float64
dtypes: float64(23)
memory usage: 2.5 MB

#To improve the accuracy of Deep learning system and machine learning system in financial analyze, I  to use PRINCIPAL #COMPONENT ANALYSIS as an input feature variable to activate the effectivness of the  model to be use in this task.
#it will improve the accuracy of the model by reduce the noise in the dataset . 
#THE IMPLEMENTATION OF PCA IS MUCH SIMPLE IN SIKICT PACKAGES LIBRARY.

In [56]:
#calculate a Principal Component Analysis on a dataset using the PCA() class in the scikit-learn library

from sklearn.decomposition import PCA

#copy the dataframe values of it float.

data = new_df.copy().values.astype('float')

# call the pca on the fatures of the dataset.
pca_featues = new_df.columns.tolist() 
#create pca class instances
pca = PCA(n_components=1)
# fit the pca on the dataset
new_df['pca'] = pca.fit_transform(data)

In [57]:
#new_df.head()
new_df.tail()

,OPEN,HIGH,LOW,CLOSE,VOL,MA,ema,momentum,roc,CCI,...,Chaikin,ADX,trix,vin,vip,TSI,aroon_down,new_aroon,OBV,pca
13848,0.817531,0.814078,0.814580,0.815418,0.796581,0.0,0.0,0.553335,0.561988,0.123308,...,0.428329,0.451315,0.898339,0.0,0.796995,0.705573,0.750,0.833333,0.101651,0.527132
13849,0.815317,0.812338,0.813959,0.815996,0.768661,0.0,0.0,0.553335,0.561988,0.123733,...,0.492380,0.438284,0.896729,0.0,0.778279,0.699864,0.725,0.816667,0.898349,0.526411
13850,0.815913,0.814033,0.816293,0.817135,0.768661,0.0,0.0,0.553335,0.561988,0.122499,...,0.519762,0.426463,0.894979,0.0,0.769032,0.695620,0.700,0.800000,0.884396,0.523874
13851,0.817069,0.814470,0.816728,0.816859,0.821083,0.0,0.0,0.553335,0.561988,0.122357,...,0.503370,0.415106,0.893089,0.0,0.790186,0.691368,0.675,0.783333,0.089408,0.515432
13852,0.816997,0.813382,0.818051,0.816921,0.017664,0.0,0.0,0.553335,0.561988,0.122285,...,0.487225,0.415106,0.891070,0.0,0.811272,0.687526,0.650,0.766667,0.910592,0.437781


In [58]:
# creating the signal trend on the closing prices. Try to predict it movement by looking back as far as 15hrs .to check if 
# return of  closing price is an upward or downward movement of prices.
# Set the signal as 1 and 0 which implies upward or downward .
new_df['Return'] = new_df['CLOSE']-new_df['CLOSE'].shift(1)
new_df['Signal'] = np.where(new_df.Return>=0,1,0)
df = new_df.fillna(0)




In [59]:
df.head()

,OPEN,HIGH,LOW,CLOSE,VOL,MA,ema,momentum,roc,CCI,...,trix,vin,vip,TSI,aroon_down,new_aroon,OBV,pca,Return,Signal
0,0.259826,0.258444,0.261347,0.263745,0.768091,0.718585,0.717667,0.523820,0.528978,0.636702,...,0.690925,0.0,0.0,0.480805,0.0,0.0,0.500000,-0.643623,0.000000,0
1,0.263784,0.261460,0.265946,0.266929,0.622792,0.718491,0.717602,0.513802,0.519473,0.632385,...,0.690925,0.0,0.0,0.480805,0.0,0.0,0.884112,-0.645674,0.003184,1
2,0.267136,0.262076,0.267854,0.266636,0.626211,0.718366,0.717538,0.524924,0.520022,0.631494,...,0.690925,0.0,0.0,0.480805,0.0,0.0,0.186788,-0.650900,-0.000293,0
3,0.266638,0.261398,0.267402,0.266173,0.603419,0.718276,0.717550,0.529879,0.532648,0.632129,...,0.690925,0.0,0.0,0.480805,0.0,0.0,0.198178,-0.654347,-0.000462,0
4,0.266158,0.262442,0.268103,0.266102,0.537322,0.718202,0.717586,0.529044,0.536743,0.631462,...,0.690925,0.0,0.0,0.480805,0.0,0.0,0.231207,-0.659838,-0.000071,0


In [60]:
df.shape

(13853, 26)

In [61]:
# creating dataset.
# look_back  = 30*30minutes = 900minutes represent 15hrs,I look_back 15hrs to predict 30minutes movement of stock prices.
def create_df(df, look_back=30):
    dataX, dataY = [], []
    for i in range(len(df)-look_back-1):
        a = df[i:(i+look_back)]
        dataX.append(a)
        dataY.append(df[i + look_back])
    return np.array(dataX), np.array(dataY)

In [63]:
# preparing a dataset for deep learning model
 #create datasets
target = df.columns.tolist().index('Signal')
df = df.values
X,y = create_df(df,look_back=30)
y = y[:,target]

In [64]:
# Set training data size
#Large enough dataset so it is divided into 80% training,10% validation and 10% test
data_train = int(len(X)*0.8)
x_train = X[:data_train]
y_train = y[:data_train]
x_test = X[data_train:]
y_test = y[data_train]           

In [65]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Input, LSTM, Dense
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [66]:
#SET LSTMs network
model = Sequential()
model.add(LSTM(30, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(30, return_sequences=True))
model.add(LSTM(20, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(8, return_sequences=False))
# output layer
model.add(Dense(8, init ='uniform', activation='relu'))
model.add(Dense(1, init ='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 30)            6840      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 20)            4080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 20)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 8)                 928       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total para

c:\users\1\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  if __name__ == '__main__':
c:\users\1\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


In [67]:
# the best weight is suppose to be save during training
# Save the best weight during training.
#from keras.callbacks import ModelCheckpoint
#checkpoint = ModelCheckpoint("weights.best.hdf5", monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')

# Fit
#callbacks_list = [checkpoint]
#history = model.fit(x_train, y_train, epochs=200, batch_size=500, verbose=0, callbacks=callbacks_list, validation_split=0.1)


In [68]:
model.fit(x_train,y_train,epochs=500,batch_size =600,verbose=1,validation_split=0.1)

Train on 9951 samples, validate on 1106 samples
Epoch 1/500
9951/9951 [==============================] - 9s 858us/step - loss: 0.6929 - acc: 0.5264 - val_loss: 0.6925 - val_acc: 0.5307
Epoch 2/500
9951/9951 [==============================] - 4s 422us/step - loss: 0.6922 - acc: 0.5268 - val_loss: 0.6918 - val_acc: 0.5307
Epoch 3/500
9951/9951 [==============================] - 4s 412us/step - loss: 0.6919 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 4/500
9951/9951 [==============================] - 4s 404us/step - loss: 0.6918 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 5/500
9951/9951 [==============================] - 4s 403us/step - loss: 0.6918 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 6/500
9951/9951 [==============================] - 4s 417us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 7/500
9951/9951 [==============================] - 4s 417us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 

9951/9951 [==============================] - 4s 410us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 61/500
9951/9951 [==============================] - 4s 396us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 62/500
9951/9951 [==============================] - 4s 419us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 63/500
9951/9951 [==============================] - 4s 404us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 64/500
9951/9951 [==============================] - 4s 432us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 65/500
9951/9951 [==============================] - 4s 427us/step - loss: 0.6918 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 66/500
9951/9951 [==============================] - 4s 399us/step - loss: 0.6917 - acc: 0.5268 - val_loss: 0.6913 - val_acc: 0.5307
Epoch 67/500
9951/9951 [=======================

9951/9951 [==============================] - 4s 398us/step - loss: 0.6884 - acc: 0.5451 - val_loss: 0.6889 - val_acc: 0.5380
Epoch 120/500
9951/9951 [==============================] - 4s 400us/step - loss: 0.6874 - acc: 0.5497 - val_loss: 0.6867 - val_acc: 0.5479
Epoch 121/500
9951/9951 [==============================] - 4s 397us/step - loss: 0.6869 - acc: 0.5476 - val_loss: 0.6857 - val_acc: 0.5407
Epoch 122/500
9951/9951 [==============================] - 4s 432us/step - loss: 0.6870 - acc: 0.5428 - val_loss: 0.6827 - val_acc: 0.5461
Epoch 123/500
9951/9951 [==============================] - 4s 404us/step - loss: 0.6882 - acc: 0.5528 - val_loss: 0.6859 - val_acc: 0.5678
Epoch 124/500
9951/9951 [==============================] - 4s 396us/step - loss: 0.6900 - acc: 0.5321 - val_loss: 0.6911 - val_acc: 0.5307
Epoch 125/500
9951/9951 [==============================] - 4s 398us/step - loss: 0.6912 - acc: 0.5298 - val_loss: 0.6905 - val_acc: 0.5371
Epoch 126/500
9951/9951 [================

9951/9951 [==============================] - 4s 401us/step - loss: 0.6823 - acc: 0.5638 - val_loss: 0.6740 - val_acc: 0.5732
Epoch 179/500
9951/9951 [==============================] - 4s 406us/step - loss: 0.6814 - acc: 0.5620 - val_loss: 0.6769 - val_acc: 0.5542
Epoch 180/500
9951/9951 [==============================] - 4s 416us/step - loss: 0.6817 - acc: 0.5560 - val_loss: 0.6744 - val_acc: 0.5552
Epoch 181/500
9951/9951 [==============================] - 4s 400us/step - loss: 0.6825 - acc: 0.5556 - val_loss: 0.6760 - val_acc: 0.5741
Epoch 182/500
9951/9951 [==============================] - 4s 392us/step - loss: 0.6823 - acc: 0.5565 - val_loss: 0.6739 - val_acc: 0.5805
Epoch 183/500
9951/9951 [==============================] - 4s 391us/step - loss: 0.6816 - acc: 0.5593 - val_loss: 0.6769 - val_acc: 0.5732
Epoch 184/500
9951/9951 [==============================] - 4s 396us/step - loss: 0.6809 - acc: 0.5641 - val_loss: 0.6759 - val_acc: 0.5705
Epoch 185/500
9951/9951 [================

9951/9951 [==============================] - 4s 387us/step - loss: 0.6793 - acc: 0.5622 - val_loss: 0.6776 - val_acc: 0.5461
Epoch 238/500
9951/9951 [==============================] - 4s 393us/step - loss: 0.6777 - acc: 0.5701 - val_loss: 0.6748 - val_acc: 0.5678
Epoch 239/500
9951/9951 [==============================] - 4s 402us/step - loss: 0.6772 - acc: 0.5674 - val_loss: 0.6721 - val_acc: 0.5859
Epoch 240/500
9951/9951 [==============================] - 4s 452us/step - loss: 0.6752 - acc: 0.5747 - val_loss: 0.6844 - val_acc: 0.5416
Epoch 241/500
9951/9951 [==============================] - 4s 402us/step - loss: 0.6759 - acc: 0.5739 - val_loss: 0.6773 - val_acc: 0.5588
Epoch 242/500
9951/9951 [==============================] - 4s 401us/step - loss: 0.6772 - acc: 0.5714 - val_loss: 0.6733 - val_acc: 0.5660
Epoch 243/500
9951/9951 [==============================] - 4s 403us/step - loss: 0.6787 - acc: 0.5663 - val_loss: 0.6747 - val_acc: 0.5606
Epoch 244/500
9951/9951 [================

9951/9951 [==============================] - 4s 399us/step - loss: 0.6665 - acc: 0.5950 - val_loss: 0.6850 - val_acc: 0.5515
Epoch 297/500
9951/9951 [==============================] - 4s 395us/step - loss: 0.6654 - acc: 0.6014 - val_loss: 0.6849 - val_acc: 0.5425
Epoch 298/500
9951/9951 [==============================] - 4s 391us/step - loss: 0.6667 - acc: 0.5946 - val_loss: 0.6900 - val_acc: 0.5407
Epoch 299/500
9951/9951 [==============================] - 4s 395us/step - loss: 0.6733 - acc: 0.5827 - val_loss: 0.6795 - val_acc: 0.5524
Epoch 300/500
9951/9951 [==============================] - 4s 395us/step - loss: 0.6683 - acc: 0.5894 - val_loss: 0.6821 - val_acc: 0.5714
Epoch 301/500
9951/9951 [==============================] - 4s 397us/step - loss: 0.6683 - acc: 0.5896 - val_loss: 0.6854 - val_acc: 0.5533
Epoch 302/500
9951/9951 [==============================] - 4s 402us/step - loss: 0.6658 - acc: 0.5979 - val_loss: 0.6853 - val_acc: 0.5588
Epoch 303/500
9951/9951 [================

9951/9951 [==============================] - 4s 450us/step - loss: 0.6492 - acc: 0.6297 - val_loss: 0.7090 - val_acc: 0.5533
Epoch 356/500
9951/9951 [==============================] - 4s 446us/step - loss: 0.6462 - acc: 0.6353 - val_loss: 0.7169 - val_acc: 0.5497
Epoch 357/500
9951/9951 [==============================] - 5s 478us/step - loss: 0.6463 - acc: 0.6308 - val_loss: 0.7033 - val_acc: 0.5425
Epoch 358/500
9951/9951 [==============================] - 5s 454us/step - loss: 0.6523 - acc: 0.6193 - val_loss: 0.7017 - val_acc: 0.5362
Epoch 359/500
9951/9951 [==============================] - 4s 450us/step - loss: 0.6488 - acc: 0.6282 - val_loss: 0.7074 - val_acc: 0.5561
Epoch 360/500
9951/9951 [==============================] - 4s 403us/step - loss: 0.6457 - acc: 0.6334 - val_loss: 0.7093 - val_acc: 0.5425
Epoch 361/500
9951/9951 [==============================] - 4s 412us/step - loss: 0.6476 - acc: 0.6292 - val_loss: 0.7036 - val_acc: 0.5416
Epoch 362/500
9951/9951 [================

9951/9951 [==============================] - 4s 410us/step - loss: 0.6381 - acc: 0.6480 - val_loss: 0.7323 - val_acc: 0.5425
Epoch 415/500
9951/9951 [==============================] - 4s 417us/step - loss: 0.6231 - acc: 0.6634 - val_loss: 0.7471 - val_acc: 0.5335
Epoch 416/500
9951/9951 [==============================] - 4s 452us/step - loss: 0.6170 - acc: 0.6693 - val_loss: 0.7583 - val_acc: 0.5407
Epoch 417/500
9951/9951 [==============================] - 4s 419us/step - loss: 0.6295 - acc: 0.6592 - val_loss: 0.7424 - val_acc: 0.5371
Epoch 418/500
9951/9951 [==============================] - 4s 406us/step - loss: 0.6233 - acc: 0.6646 - val_loss: 0.7461 - val_acc: 0.5253
Epoch 419/500
9951/9951 [==============================] - 4s 389us/step - loss: 0.6104 - acc: 0.6792 - val_loss: 0.7517 - val_acc: 0.5479
Epoch 420/500
9951/9951 [==============================] - 4s 391us/step - loss: 0.6132 - acc: 0.6754 - val_loss: 0.7531 - val_acc: 0.5172
Epoch 421/500
9951/9951 [================

9951/9951 [==============================] - 4s 384us/step - loss: 0.5765 - acc: 0.7063 - val_loss: 0.8135 - val_acc: 0.5389
Epoch 474/500
9951/9951 [==============================] - 4s 394us/step - loss: 0.5732 - acc: 0.7070 - val_loss: 0.7821 - val_acc: 0.5425
Epoch 475/500
9951/9951 [==============================] - 4s 405us/step - loss: 0.5587 - acc: 0.7246 - val_loss: 0.8257 - val_acc: 0.5344
Epoch 476/500
9951/9951 [==============================] - 4s 407us/step - loss: 0.5527 - acc: 0.7290 - val_loss: 0.8130 - val_acc: 0.5380
Epoch 477/500
9951/9951 [==============================] - 4s 398us/step - loss: 0.5673 - acc: 0.7160 - val_loss: 0.8411 - val_acc: 0.5344
Epoch 478/500
9951/9951 [==============================] - 4s 395us/step - loss: 0.5691 - acc: 0.7129 - val_loss: 0.8032 - val_acc: 0.5271
Epoch 479/500
9951/9951 [==============================] - 4s 407us/step - loss: 0.5812 - acc: 0.7010 - val_loss: 0.8120 - val_acc: 0.5434
Epoch 480/500
9951/9951 [================

In [69]:
# put the model into use,making a prediction for target  of the tets size
y_pred = model.predict(x_test)


In [70]:
#FIND THE PROBABILITY OF THE PREDICTION ON CLASS SIGNAL
ynew = [0 if x < 0.5 else 1 for x in y_pred]
ynew  

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [71]:
# import the modules from sklearn.metrics

from sklearn.metrics import confusion_matrix,precision_score,recall_score
# confusion matrix
confusion_matrix(y_test,ynew)

TypeError: Singleton array 0.0 cannot be considered a valid collection.